# Hands-on Machine LEarning with Scikit-learn, Keras & Tensorflow

## Aurelien Geron

## Second EDITION

### OREILLY


In [ ]:
# Contain some of our base function and attributes
from config import * 

import pandas as pd
import joblib
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, cross_val_score, GridSearchCV
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV
# Ipython magic line which indicate the graphs depends on Matplotlib as backend for displaying
%matplotlib inline 



In [ ]:
_ = fetch_housing_data() # The function used from config file
df_file = load_housing_data() # The function used from config file

In [ ]:
df_file.head() # Display first 5 rows of the Dataframe table of the file we load it

# Info() Method

The info() method help us to get intuition about each column(attribute) in our dataset either in:
-  The data type of each column.
- The number of Non-Null values that this column has(overall cell that not null).
- The memory that this file takes from our RAM.

In [ ]:
df_file.info() 

# value_counts() Method

When it comes to categorical variables which hold some of the discrete values like classification problem, in general 
you have number of classes like all cell in this columns will have one of these values either [0,1,2], so you have three classes, but what about the cell that contain **class 1** then the *value counts method* will count this for each class. 

In [ ]:
# You can see the discrete categorical in the column ocean_proximity on left side and their counts on right side
df_file.ocean_proximity.value_counts()

# describe() Method

In statics we have different measurement of analysis, which get your intuition about the data you dealing with like the average  of the data [2, 3, 1,4, 3, 5], so you will first rearrange them not for the mean but for other statics calculation, so it will be [1,2,3, 3,4,5] then the mean(average) = (1+2+3+3+4+5) = 18 /  how many numbers you summation = 18/6 = 3, also the **median** will be 3 the mode will be 3, other values we are trying to get is which called **first Quarter Range**, **IQR** and **Third Quarter Range**, which here repsendted as 25%, 50%, 75%.

- Others like Standard Deviation which the spread of data points from the mean value.
- min = the minium value in this columns.
- count = the number of non-null cells in this column
- max = the max value in this columns

# Note !

You should be noticed that there are just 9 columns even of the dataset is contain 10 columns but this because should we know these statistics are only worked on numbers.

In [ ]:
df_file.describe()

# hist() Method

The histogram method give you a quick review of the values in your data and their frequency.
This is helpful for a quick overview of how the most frquent range of values, also about the maxmium and minum of ranges, display also which columns need to hande the features scling for.
It also suggests questions for you to look for just like:

The housing median age and the median house value were also capped. The latter may be a serious problem since it is your target attribute (your labels). Your Machine Learning algorithms may learn that prices never go beyond that limit. You need to check with your client team (the team that will useyour system’s output) to see if this is a problem or not. If they tell you that they need precise predictions even beyond $500,000, then you have mainly two options:

- Collect proper labels for the districts whose labels were capped.
- remove those districts from the training set (and also from the test set, since your system shouldnot be evaluated poorly if it predicts values beyond $500,000)

In [ ]:
df_file.hist(bins=20, figsize=(15, 15))

# NOTES !!

We can noticed some of the points that related to graphs above like the different scales that the attributes have and for machine learning its better to pass the attributes at some ranges, what we called features scaling.

Also the mendain_income attribute is from .5 to 15 and its ambigous so you need to check this with the owner to know how it going from maybe it means 15,000 or multiply by spesific value.

Also, the price of the house if should go beyond the max and min values or not, and the ranges that have a small number of instances should be removed or not because maybe it causes misleading for the model.

Some of the attributes have skewed right graphs, why?

All of these are questions you can extract from the graphs to ask and trying to find answers.

In [ ]:
# Call the hist method on just one column
df_file['total_bedrooms'].hist(bins=20, figsize=(7, 7))

# Create a Test Data

This is not easy step or stage, its not actually about split the data, but its about how you ensure that your train and test dataset are included all of the different spectrum of the data which means not bias to some category of the data.


In [ ]:
train_set, test_set = split_train_test(df_file, .2)

In [ ]:
print(len(train_set))
print(len(test_set))

In [ ]:
train_set[:3] # see that first unnamed column have the random indices

# Note !

The problem here is that every time you will run you will have another test and train set which after you train your model with time your model will train overall the data as it with a time its see the test data, but to avoid this either to save your test data in separate file or to make a **seed**, and its will generate the same shuffle indices.

In [ ]:
# using sklearn library
train_set, test_set = train_test_split(df_file, test_size=.2, random_state=42)
print(len(train_set))
print(len(test_set))

# Size of the dataset

When the dataset are large enough relative to number of attributes(features), it okay to consider these random method, but if not, you run risk of introducing a signigcant sampling **bias**.

In this case we use the method called ** Stratified Random Sampling **, is a method of sampling that involves the division of a population into smaller sub-groups known as strata, the strata are formed based on members shared attributes or characteristics such as income or educational attainment.

Stratified random sampling is also called proportional random sampling or quota random sampling. 

Stratified random sampling allows researchers to obtain a sample population that best represents the entire population being studied.


In [ ]:
# Call the hist method on just one column
df_file['median_income'].hist(bins=20, figsize=(7, 7))

We can see the problem above in this example, let us imagine that you have split the dataset to train and test randomly using some of straightforward method, as we can see may be your split will take all random instance in the train between 1 to 8 since most of the median income and and test the same was happend except a few example from medain income above 8, but like these problems will be baised since most of your instance are from range 1 to 8, also what happen if it split most of your train in range from 1 to 8 and you find that the test have medain income from 1 to 14, it will misleading your model.

**So its important ot have a sufficient number of instance(rows) in your dataset for each stratum. Or it will cause to bais**

In [ ]:
df_file['income_cat'] = pd.cut(df_file['median_income'],
                              bins=[0., 1.5, 3.0, 4.5, 6., np.inf], 
                              labels=[1, 2, 3, 4, 5])
# df_file['income_cat']

In [ ]:
df_file['income_cat'].hist()

Here is we can see that its create a new attribute with 5 categories from 1 to 5 and each category have range like category 1 have range from 0 to 1.5 and category 2 have range from 1.5 to 3.

**Since the median income attribute have its own effect on the price of the house we can make this variable is resposible for our spliting to the dataset instead of the simple methods.**

In [ ]:
def measure_income_compare(data):
    class_list = [0, 0, 0, 0, 0]
    for i in data['median_income']:
        if i >= 0 and i < 1.5:
            class_list[0] +=1
        elif i >= 1.5 and i < 3:
            class_list[1] +=1
        elif i >= 3 and i < 4.5:
            class_list[2] +=1
        elif i >= 4.5 and i < 6:
            class_list[3] +=1
        else: class_list[4] +=1
    return class_list

In [ ]:
split = StratifiedShuffleSplit(n_splits=1, test_size=.2 , random_state=42)
for train_index, test_index in split.split(df_file, df_file['income_cat']):
    strat_train_set = df_file.loc[train_index] # retrive rows with these indces
    strat_test_set = df_file.loc[test_index] # retrive rows with these indces

In [ ]:
strat_train_set.head()

In [ ]:
# The number of each category like from 0 to 1.5 about some precentage
stratified = np.array(strat_test_set['income_cat'].value_counts() / len(strat_test_set))
random = np.array(measure_income_compare(test_set))/len(test_set)
overall = np.array(measure_income_compare(df_file))/ len(df_file)

In [ ]:
stratified.sort()
random.sort()
overall.sort()
compare_dict = {'overall': overall, 'stratified': stratified, 'random': random}
compare_df = pd.DataFrame(compare_dict)

# In table below we can see how stratified static method most similar to overall coverage of our dataset
# look also at the simple random Vs overall
compare_df['Stratified. %error'] = np.abs((compare_df['stratified'] - compare_df['overall']) * 100)
compare_df['Random. %error'] = np.abs((compare_df['random'] - compare_df['overall']) * 100)
compare_df

In [ ]:
strat_train_set.head()

In [ ]:
# Now remove income cat to back data to its original form
strat_train_set.drop("income_cat", axis=1, inplace=True)
strat_test_set.drop("income_cat", axis=1, inplace=True)
strat_train_set.head()

# Discover and Visualize the data to gain insights

With some of the analysis above we get a quick intuation about the dataset, but more to go in depth of understanding the data we dealing with, we need to discover these data more than this quick intuation, and the graphs are the most common to discover the data since our brain is very good spotting patterns in pictures.

First keep your test set aside and make the discover on the training and also take a copy to keep the original training set unchanged, beside of that if the training data is large just look into sample of the data to consider time and memory usage.
## Visualizing Geographic Data

Since there is a geographical information (longtuide and latuide), it's helpful to know about which area have the most house or the density of your data in which are, it may help you to know also why price in some area are greater than the other areas.

In [ ]:
df_file.plot(kind="scatter", x="longitude", y="latitude")

This will give you a littile information,because you have not see any pattern of information.

Then in like these cases the paramters of the graph can help a lot to help you go deeper in discover process.

- What about change the opacity of the circle of scatter plot




In [ ]:
df_file.plot(kind="scatter", x="longitude", y="latitude", alpha=.2)

**It help you to know the density by overlapping the circle if this area have a lot of house there, also other area that have small number of house will be appear because of its opacity will be low.**

- Not just alpha, other parameters of the graph can help you discover more pattern

In [ ]:
df_file.plot(kind="scatter", x="longitude", y="latitude", alpha=.3,
            s=df_file['population']/50, label="population", figsize=(10,5),
            c="median_house_value", cmap=plt.get_cmap('jet'), colorbar=True)
plt.legend()

# Parameters

- s, is the size of the data point, and s=df_file['population']/50, it help us know which area has most density.
- c, is the color that represent the price, which represent range from blue(low price) to red (high price) as we can see in the colorbar.
- cmap, color map which replace the default (black and white) of c, to blue and red.
- colorbar, true to represent the range of color to notice explain the point on the graph.

**From the graph you can now get a clear intuition about how the location affect the price of the house., not just the location also the population density**


# Looking For correlation

As we know our target variables, it's helpful to get the correlation of this target variables against your attributes.

In [ ]:
corr_matrix = df_file.corr()
corr_matrix['median_house_value'].sort_values(ascending=False)

We can see that median_house_value is so correlated with it self and from this point we should noticed that the values are in range from -1 to 1 and 1 which represent positive higher dpendancy, between that the relationship are close to be or no as in independent variables with 0 correlation or -1 when its strong negative relationship.

Actually the correlation is like the linear equation between x & y, if its 1 then x & y are equal which represent a straight line and for other values it will be different.

**Now maybe we need to get close to median income and graph it against median_house_value as it make the most affect on the price Or we can display scatter_matrix for the most three or four attributes againsit each other**

In [ ]:
# Scatter_matrix 
pd.plotting.scatter_matrix(df_file[['median_house_value', 'median_income', 'total_rooms']], figsize=(10, 7))

In [ ]:
# get close to median house value against median income
df_file.plot(kind="scatter", x="median_income", y="median_house_value", alpha=.1)

# Note !

**This attribute median income have a lot of affect on the price of the house as we can see even with small alpha, not just that, it also can represent some of the horizontal lines at 500,000 and between 3000,00 and 4000, 00, and this may indicate that there is no affect of the median income when in either small or large price the house value is still large, and in this case may be we need to remove like these points from our dataset because it may cause the learning algorithm to overfitting or misleading learning**.

# Attribute Combination

The most helpful thing of machine learning is that it accept the creativity of your solution not just what the data present you should use, no you have the ability to improve the dataset maybe remove some attribute as it has no effect, combine two attributes via math equation, or create new attribute from existing ones.

- median_house_value    1.000000
- median_income         0.688075
- total_rooms           0.134153
- housing_median_age    0.105623
- households            0.065843
- total_bedrooms        0.049686
- population           -0.024650
- longitude            -0.045967
- latitude             -0.144160


We can see here the correlation of the previous stage and if we noticed that the total_rooms can help in finding the number of rooms per house not total room per households, also total_bedrooms and population can help us create a new correlation between attributes.


In [ ]:
df_file['rooms_per_household']     = df_file['total_rooms']    / df_file['households']
df_file['bedrooms_per_room']       = df_file['total_bedrooms'] / df_file['total_rooms']
df_file['pupulation_per_ousehold'] = df_file['population'] / df_file['households']

In [ ]:
# Now look at the new correlation and find new insights
corr_matrix = df_file.corr()
corr_matrix['median_house_value'].sort_values(ascending=False)

In [ ]:
df_file.head() # look at the dataset now

# Some new insights

- Bedrooms_per_room decrease with increasing of median_house_value (So strong correlation).
- Rooms_per_household, increase the price of the house by increasing the rooms per house.


# Prepare Data for Machine Learning Algorithm

We doing a pipeline for this process to use with future work or similar data or with the training and testing data or even on the system when we lunch model to work on, we need the new data to feed in this pipeline.

**We will separate the target variables from the other attributes**

In [ ]:
# drop median_house_value and save rest of strat_train_set into df_file
df_file = strat_train_set.drop("median_house_value", axis=1) 
df_file_labels = strat_train_set['median_house_value'].copy() 

# Data Cleaning

There are three option to handle missing values(Nan) values:
- Get rid of corresponding rows
- Get rid of the attribute itself
- Replace with meaningful value, in numbers may be median, mode, zero, and other suitable static based on the problem.

In [ ]:
# only one attribute has missing values which is total_bedrooms
df_file.dropna(subset=["total_bedrooms"])
df_file.drop("total_bedrooms", axis=1)
median = df_file['total_bedrooms'].median()
df_file['total_bedrooms'].fillna(median, inplace=True)

# Note !

When we use option 3, we should save the median for test dat and other new data when system go live.

Also, we can use the sklearn impute library to handle missing values.

Also, we will rid the one-categorical attribute because the imputer can not deal with text once you pass static median. and we make this process on all the numbers attributes , because maybe in the large dataset which has a lot of attributes it will be difficult to check each attribute or even when the system go life maybe some of attributes can returned without values.

In [ ]:
imputer = SimpleImputer(strategy="median")
df_file_num = df_file.drop("ocean_proximity", axis=1)
imputer.fit(df_file_num)
# As we save our median for future sklearn imputer handle this and save it for you in statistics_ attribute 
imputer.statistics_ 

In [ ]:
# To ensure of these values calculated by sklearn
df_file_num.median()

In [ ]:
X = imputer.transform(df_file_num) # the result is numpy array
df_file_tr = pd.DataFrame(X, columns=df_file_num.columns, index=df_file_num.index)
df_file_tr.head()

# Handling Text and Categorical Attributes

After we have looked at the numerical attributes we also need to take a look on the text or categorical attributes, because machine learning models can not deals with text.

In [ ]:
df_file_cat = df_file['ocean_proximity']
print(type(df_file_cat))
df_file_cat = df_file[['ocean_proximity']]
print(type(df_file_cat))
df_file_cat.head(10)

**We can noticed that, it's a categorical attribute not just any text, and its repeated with same case and characters, we can use machine learning OrdinalEncoder class to deal with this categorical attribute.**

In [ ]:
ordinal_encoder = OrdinalEncoder()
df_file_cat_encoded = ordinal_encoder.fit_transform(df_file_cat)
print(df_file_cat_encoded[:10])
# _ means learned parameters while **categories** without _ means hyperparameters
print(ordinal_encoder.categories_) 

**In fitting categorical data should we know it's ordinal category or not, because like this case as it not ordicanal the ml may learn pattern from that, and we do not need like this to happen**

The solution in this case is to use binary representation for each category with 1 to such appearing category and 0 for the others, so it will be vector of 4 numbers * the number of the dataset.

**Sklearn also has onehotencoder method to handle this.**

In [ ]:
one_hot_encoder = OneHotEncoder()
df_file_cat_1hot = one_hot_encoder.fit_transform(df_file_cat)
df_file_cat_1hot

# Sparse Matrix

Because the one-hot encoded method give 1 for which category and 0 for others, this will generate a sparse matrix with most of 0, since of that sklearn save only the location of each 1 in the matrix and ignore the zeros, but we can retrieve the whole matrix using **toarray** method associated with the df_file_cat_1hot object.

In [ ]:
df_file_cat_1hot.toarray()

In [ ]:
one_hot_encoder.categories_

# Custom Transformer

Everything has an end and the library you use maybe it provide you with a lot of helpful APIs, but sometimes you need to create your transformer, which based on the analysis you made.

And to combine this with sklearn it helpful to apply it in the form that sklearn do, as [fit, transform, fit_transform].

Also you can have other methods or attributes as you need, maybe you have another function of cleaning and you need to make a pipeline for each of stage you have.

Other than that you can combine attributes of hyperparameters for tuning.


In [ ]:
attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
df_file_extr_attribs = attr_adder.transform(df_file.values) # as numpy array

# Features Scaling

One of the most interesting and helpful stages is the feature scaling when the dataset attributes go to different ranges like some attribute between [1-100], others [1000, 10000000], and others are different,  in this case of numerical ranges of different attributes, it's helpful to standardize all the numeric attributes to specific range always [0-1], which applied by the Max-Min function, which divides each value by max minus min which will restrict all values to be between 0 and 1, other function is using standard deviation and mean, but its goes to the different distribution of ranges. 

This is help machine learning to perform well in this case of **Features Scaling**, also some times when the model in learning if we do not apply **Features Scaling** and values of some attributes are large, this can cause explode of values in the iteration when the model trying to minimize the cost function.


# Transformation Pipelines

As we can see, we have passed through different data transformation stages, because of that we should care of the order these transformer are applied to the data, So we can use the idea of **PipeLine** either create your own transformer function or use sklearn pipeline.


In [ ]:
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('attribs_addder', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler()),
])
df_file_num_tr = num_pipeline.fit_transform(df_file_num)
df_file_num_tr

# PipeLine

The pipeline constructor of sklearn takes a list of tuples each of these tables are pairs of **name/estimator**, the Last table should contain transformer estimator.

When we cal the fit() method of Pipeline object, it calls fit_transform() **fit_transform() of object in the pipeline** methods sequentially on all transformers, then it pass the result output of each call to next call until the end of pipeline

# Numerical & Categorical Attributes

We also need to make our pipeline to handle the two different types of data Numerical and Categorical, so we designed another pipeline that works with all columns (attributes).

Also, sklearn has **ColumnTransformer** that we can feed the full pipeline to and it works fine with pandas data frames.

In [ ]:
num_attribs = list(df_file_num)
num_attribs

In [ ]:
cat_attribs = ["ocean_proximity"]
df_file.head()

In [ ]:
full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', OneHotEncoder(), cat_attribs),
])
df_file_prepared = full_pipeline.fit_transform(df_file)

# Select and Train a Model

Now we are ready to select and train the Machine Learning Model, after all these previous steps, now the rest is simple than what we have before.

Just we can import and use linear regression model, lets do it.


In [ ]:
df_file_prepared.shape

In [ ]:
df_file.head()

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(df_file_prepared, df_file_labels)

In [ ]:
# Now its done we fit the model we can test some prediction and see the score
some_data = df_file.iloc[:5]
some_labels = df_file_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)
print("Prediction: ", lin_reg.predict(some_data_prepared))
print("Real Lables: ", list(some_labels))

In [ ]:
# Now lets do it on the whole dataset
df_file_predictions = lin_reg.predict(df_file_prepared)
lin_mse = mean_squared_error(df_file_predictions, df_file_labels)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

# Underfitting
As we can see how its bad prediction since of this large error, and as we talked in chapter 1 its underfitting problem, which the model have not fit the data well, and this may back to number of things:

- The Model is so simple to get pattern from the data, or to fit the data as it linear equation.
- The dataset itself may contain insufficient attributes to let the model learn from.
- insufficient quantity of the training data.
- Regularization part if we have.
- irrelevant features(attributes), which can mean that features does not provide the model with enough information to make a good prediction.


# DecisionTreeRegressor

Now we are going to use more complex model which can find a nonlinear relationship in the data.

In [ ]:
tree_reg = DecisionTreeRegressor()
tree_reg.fit(df_file_prepared, df_file_labels)
df_file_predictions = tree_reg.predict(df_file_prepared)
tree_mse = mean_squared_error(df_file_predictions, df_file_labels)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

# Overfitting

As we also talked about earlier in the previous chapter the model may be overfitting your training data, and for that, we kept aside part of the data which the test set to test our model on unseen examples, and see how it works,the model has no error on the training data, but what if we have predicted the test data.

But as we also know it's helpful to cut part of training data as it called (dev-set) for validating the model, because we should use the test data when we are ready to launch the model to be confident of this model.



# Better Evaluation Using Cross-Validation

We can use sklearn train_test_split function to split the training set into train and validation sets, then train model on this training set and evaluate it on validation set.

But a better way is to use K-Cross-Validation method, which help you to train and validate your model on the whole training set, since it devide your training set into K flods, use one of them to evalute as validation set and K-1 to train on, next iteration use another flod and repeat the process to end of K-foods which have trained and evaluated on.


In [ ]:
scores = cross_val_score(tree_reg, df_file_prepared, df_file_labels, cv=10,
                        scoring="neg_mean_squared_error")
tree_rmse_scores = np.sqrt(-scores)

In [ ]:
def display_scores(scores):
    print("Scores: ", scores)
    print("Mean: ", scores.mean())
    print("Standard deviation: ",scores.std())
    return True

In [ ]:
display_scores(tree_rmse_scores)

# Note !
**As we can see that the tree model is same as linear regression even of it fit well on the training data it cause overfitting problem and have fall on the validation set**

In [ ]:
lin_scores = cross_val_score(lin_reg, df_file_prepared, df_file_labels, cv=10,
                            scoring='neg_mean_squared_error')
lin_rmse_scores = np.sqrt(-lin_scores)

display_scores(lin_rmse_scores)

# RandomForestRegressor

Another model can help us to ensure that the underfitting problem is away of simpler or complex usage models, it can be one of the other problems that underfitting problems happen because of it.

**Instead of using just one Decision tree the  RandomForestRegressor trying many Decision trees on random subset of the features (attributes), then averaging out their prediction**

# Note !
Like this models **RandomForestRegressor** called **Ensemble** since it building a model on top of many other models which can push machine learning algorithm to go further than expect.

In [ ]:
forest_reg = RandomForestRegressor()
forest_reg.fit(df_file_prepared, df_file_labels)

In [ ]:
df_file_predictions = forest_reg.predict(df_file_prepared)
forest_mse = mean_squared_error(df_file_predictions, df_file_labels)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

In [ ]:
forest_scores = cross_val_score(forest_reg, df_file_prepared, df_file_labels, cv=10,
                            scoring='neg_mean_squared_error')
forest_reg_rmse_scores = np.sqrt(-forest_scores)

display_scores(forest_reg_rmse_scores)

# Note !

The score now looks better than other models, but we still have the overfitting problem as we can see how it fit well on the training, and fill in evaluation on the validation set.

We can work with overfitting problems via:

- If We can get more training data.
- We can provide the model with suitable regularization.
- Look at the number of attributes against number of training examples, if they are close we can rid of some attributes.
- Reduce the noise of the data.
- Try simpler models.

# Fine-Tune Models

This stage comes late because after we have run our pipeline, then trying different models, we have to choose which best of them even if there is more than one model, you next need to **fine-tune** the parameters of the model to make it doing best as you can, after this step of cross-validation that we have and since we choose the model, now we are in the last stage which with the chosen model we are trying to fine-tune these models with best parameters that make the model fit well then we can test our chosen model on the test set.


# Grid Search

Because each model associated with a not small number of hyperparameters which you need to tune the model using these parameters, is not a handy way to make it your self manually as a lot of time you should consider in all of the combinations of these hyperparameters.
So an alternative  solution is to use the best way **Grid Search**.

With **Grid Search** all just you need is to feed what hyperparameters you need to use and for each of them what the different values you need to feed the model with and the **Grid Search** will make all the combinations for you and provide you at the end with the best hyperparameters values.


In [ ]:
param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]}, # combination one
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]}, # combination Two
]
forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid, cv=5, 
                           scoring='neg_mean_squared_error',
                          return_train_score=True)
grid_search.fit(df_file_prepared, df_file_labels)

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.refit

In [ ]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres['mean_test_score'], cvres['params']):
    print(np.sqrt(-mean_score), params)

# Radomized Search

The grid search approach is fine but with few relative combination of hyperparameters, but when it seems to have multiple or large number of hyperparameters, and it work in same way of **Grid Search**, but instead of trying all combinations, it randomize these hyperparameters and choose some of them for each iteration.

It help you in:
- Randmoize maybe for 1000 different value for each hyperparamter, instead of few predefind.
- This work depends on your situation of sources that you have, because it takes time computing and other consumed budget, it let you to iterate the number you need.

# Ensemble Methods

Instead of test each model individually, this method combine some of the model that you see can help you them combine them together to see how they work.

# Analyze the best Models and their errors

By all of these trials you have consider which best model that you will use on the test set, and the best model you choose can indicate which attributes of the data indicate to the best accurate result.

All we need is to list best_estimator_ with their corresponding attributes.


# Evaluate Your Models on Test set

After all of these steps from pipeline on the data, to tune the model paramters and trying different models for best result, should we have choose best of these models and now its work for test the models on testset.

After looking at these values associated with each attributes, you can decide which of them will be helpful, and you can drop some of less useful of these attributes.

You should look at your system and try to understand which makes these error, and how you can solve like these errors.

In [ ]:
features_importances = grid_search.best_estimator_.feature_importances_
features_importances

In [ ]:
extra_attribs = ['rooms_per_hhold', 'pop_per_hhold', 'bedrooms_per_room']
cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_attribs + extra_attribs + cat_one_hot_attribs
sorted(zip(features_importances, attributes), reverse=True)

In [ ]:
final_model = grid_search.best_estimator_
X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set['median_house_value'].copy()
X_test_prepared = full_pipeline.transform(X_test)
final_predictions = final_model.predict(X_test_prepared)
final_mse = mean_squared_error(final_predictions, y_test)
final_rmse = np.sqrt(final_mse)
final_rmse

# Exercises


#### Q1-

Try a Support Vector Machine regressor (sklearn.svm.SVR), with various hyperparameters such as kernel="linear" (with various values for the C hyperparameter) or kernel="rbf" (with various values for the C and gamma hyperparameters). Don't worry about what these hyperparameters mean for now. How does the best SVR predictor perform?

In [ ]:
param_grid = [
    {'kernel': ["linear", "rbf"], 'C': [0.01, .1, 1], 'gamma': ['scale', 'auto']}, # combination one
]

svr_model = SVR()
grid_search = GridSearchCV(svr_model, param_grid, cv=3, 
                           scoring='neg_mean_squared_error',
                          return_train_score=True)
grid_search.fit(df_file_prepared, df_file_labels)

In [ ]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres['mean_test_score'], cvres['params']):
    print(np.sqrt(-mean_score), params)

# Q2

Try replacing GridSearchCV with RandomizedSearchCV.

In [ ]:
param_grid = [
    {'kernel': ["linear", "rbf", "sigmoid"], 'C': [0.01, .1, 1, 3, 5], 'gamma': ['scale', 'auto']}, # combination one
]

svr_model = SVR()
random_search = RandomizedSearchCV(svr_model, param_grid, n_iter=10, cv=5, 
                           scoring='neg_mean_squared_error',
                          return_train_score=True)
random_search.fit(df_file_prepared, df_file_labels)

In [ ]:
cvres = random_search.cv_results_
for mean_score, params in zip(cvres['mean_test_score'], cvres['params']):
    print(np.sqrt(-mean_score), params)

In [ ]:
param_grid = [
    {'kernel': ["linear"], 'C': np.linspace(5, 1000, 20), 'gamma': ['scale']}, # combination one
]

svr_model = SVR()
random_search = RandomizedSearchCV(svr_model, param_grid, n_iter=20, cv=3, 
                           scoring='neg_mean_squared_error',
                          return_train_score=True)
random_search.fit(df_file_prepared, df_file_labels)

In [ ]:
cvres = random_search.cv_results_
for mean_score, params in zip(cvres['mean_test_score'], cvres['params']):
    print(np.sqrt(-mean_score), params)

In [ ]:
np.sqrt(-random_search.best_score_)

In [ ]:
random_search.best_estimator_

In [ ]:
param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]}, # combination one
]
forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid, cv=5, 
                           scoring='neg_mean_squared_error',
                          return_train_score=True)
grid_search.fit(df_file_prepared, df_file_labels)

In [ ]:
features_importances = grid_search.best_estimator_.feature_importances_
features_importances

In [ ]:
preparation_and_feature_selection_pipeline = Pipeline([
    ('preparation', full_pipeline),
    ('feature_selection', SelectTopFeatures(features_importances, 5))
])

housing_prepared_top_k_features = preparation_and_feature_selection_pipeline.fit_transform(df_file)
housing_prepared_top_k_features.shape

# Q-4

Try creating a single pipeline that does the full data preparation plus the final prediction.

In [ ]:
preparation_and_feature_selection__pipeline = Pipeline([
    ('preparation', full_pipeline),
    ('forest_reg', SelectTopFeatures(features_importances, 5)),
    ('forest_reg', RandomForestRegressor(random_search.best_estimator_)),
])

housing_prepared_top_k_features = preparation_and_feature_selection_pipeline.fit_transform(df_file)
housing_prepared_top_k_features.shape

#                                    الحمدلله تم بفضل الله وكرمة 